In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split


In [3]:
project_dir = r"C:\Users\kishv\OneDrive\Desktop\flights-sagemaker-projects"
data_dir = r'Data'

In [4]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(project_dir,data_dir,file_name)
    return pd.read_csv(file_path)

    

In [5]:
flights = get_data("flight_price")
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [6]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- There are total 10683 columns and 11 features
- Columns 'Route' and 'Total_Stops' have a missing value each
- Data types of some features arent appropriate. 

### 3.) Preliminary Steps

### 3.1 Check Data Types

In [7]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [8]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

### 3.2 Check For Duplicates

In [9]:
flights[flights.duplicated(keep=False)].sort_values(["Airline","Date_of_Journey","Source","Destination"])

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observations

- The type of "Date_of_Journey", "Dep_Time", "Arrival_Time", 
- The type of "Duration" should be numeric
- "Total_stops" should be numeric type
- There are 220 duplicates. They should be removed.
- Observation index 6474 is wrong. Delete this row

### 4.) Detailed Analysis

### Airline

In [10]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Some of the values have inconsistent values

### 5.) Cleaning Operations.

In [13]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ",expand = True)
        .set_axis(['hour','minute'],axis=1)
        .assign(
        hour=lambda df_:(
        df_
        .hour
        .str.replace("h","")
        .astype(int)
        .mul(60)
        ),
        minute=lambda df_:(
            df_
            .minute
            .str.replace("m","")
            .fillna('0')
            .astype(int)
        )
    )
    .sum(axis=1)
    )

In [14]:
def clean_data(df):
    return(
        df
        .drop(index=6474)
        .drop_duplicates()
        .assign(**{
            col:df[col].str.strip()
            for col in df.select_dtypes(include="O").columns
        })
        .rename(columns= str.lower)
        .assign(
            airline = lambda df_:(
                df_
                .airline
                .str.replace(" Premium economy","")
                .str.replace(" Business","")
                .str.title()
            ),
            date_of_journey = lambda df_: pd.to_datetime(df_.date_of_journey,dayfirst = True),
            dep_time = lambda df_: pd.to_datetime(df_.dep_time).dt.time,
            arrival_time = lambda df_: pd.to_datetime(df_.arrival_time).dt.time,
            duration = lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops = lambda df_: (
                df_
                .total_stops
                .replace("non-stop","0")
                .str.replace(" stops?","",regex=True)
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info = lambda df_ : df_.additional_info.replace("No info","No Info")
        )
        .drop(columns = 'route')
    )

In [15]:
flights_cleaned = clean_data(flights)
flights_cleaned

C:\Users\kishv\AppData\Local\Temp\ipykernel_17096\1856120868.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time = lambda df_: pd.to_datetime(df_.dep_time).dt.time,
C:\Users\kishv\AppData\Local\Temp\ipykernel_17096\1856120868.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time = lambda df_: pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


### 6.) Split the Data

In [17]:
flights_final = flights_cleaned.sample(1000)
x = flights_final.drop(columns="price")
y = flights_final.price.copy()

In [18]:
x_,x_test,y_,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_,y_,test_size = 0.2,random_state=42)

print(x_train.shape, y_train.shape)
print(x_val.shape,y_val.shape)
print(x_test.shape,y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


In [19]:
def export_data(x,y,name):
    file_name = f'{name}.csv'
    file_path = os.path.join(project_dir,data_dir,file_name)
    x.join(y).to_csv(file_path,index=False)
    return pd.read_csv(file_path).head()

In [20]:
export_data(x_train,y_train,"train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-12,Delhi,Cochin,16:00:00,21:00:00,300,1.0,No Info,6069
1,Indigo,2019-03-01,Chennai,Kolkata,14:40:00,17:05:00,145,0.0,No Info,7295
2,Spicejet,2019-06-03,Mumbai,Hyderabad,05:45:00,07:15:00,90,0.0,No Info,2753
3,Jet Airways,2019-06-03,Delhi,Cochin,21:25:00,19:00:00,1295,2.0,No Info,13292
4,Jet Airways,2019-06-06,Kolkata,Banglore,20:25:00,10:55:00,870,1.0,In-flight meal not included,10539


### 7.) Export the Subset

In [186]:
(
    flights
    .Duration
    .loc[lambda ser : ~ser.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [197]:
(
    flights
    .Duration
    .loc[lambda ser : ~ser.str.contains("h")]
    # .unique()
)

6474    5m
Name: Duration, dtype: object

In [193]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


In [224]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ",expand = True)
    .set_axis(['hour','minute'],axis=1)
    .assign(
        hour=lambda df_:(
        df_
        .hour
        .str.replace("h","")
        .astype(int)
        .mul(60)
        ),
        minute=lambda df_:(
            df_
            .minute
            .str.replace("m","")
            .fillna('0')
            .astype(int)
        )
    )
    .sum(axis=1)
    .rename("duration_minutes")
    .to_frame()
    .join(flights.Duration.drop(index=6474))   
)


,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


In [239]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [249]:
(
    flights
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","",regex=True)
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

In [265]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)